In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Pre-processing

In [21]:
train_data = pd.read_csv("train.csv")
train_data = train_data.dropna()

validation_data = pd.read_csv("test.csv")
fillna_values = {'CryoSleep': False, 
                 'VIP': False,
                 'Age': 0,
                 'VRDeck': 0,
                 'Cabin': '/0/',
                 'Destination': '',
                 'ShoppingMall': 0,
                 'Name': '',
                 'RoomService': 0,
                 'FoodCourt': 0,
                 'HomePlanet': '',
                 'Spa': 0}
validation_data.fillna(value=fillna_values, inplace=True)

print(len(train_data))
print(len(validation_data))

6606
4277


/var/folders/gk/5wp3t1bs4g31vy6yjwxt0zyw0000gn/T/ipykernel_1136/1271862923.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  validation_data.fillna(value=fillna_values, inplace=True)


In [22]:
# Identify rows with any NaN values
rows_with_nan = validation_data[validation_data.isna().any(axis=1)]
print(rows_with_nan)

Empty DataFrame
Columns: [PassengerId, HomePlanet, CryoSleep, Cabin, Destination, Age, VIP, RoomService, FoodCourt, ShoppingMall, Spa, VRDeck, Name]
Index: []


In [23]:
train_data.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,6606.000000,6606.000000,6606.000000,6606.000000,6606.000000,6606.000000
mean,28.894036,222.991674,478.958523,178.356494,313.161520,303.780048
std,14.533429,644.987936,1678.592291,576.328407,1144.016291,1127.142166
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,49.000000,82.750000,30.000000,65.000000,52.000000
max,79.000000,9920.000000,29813.000000,12253.000000,22408.000000,20336.000000


In [24]:
train_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [25]:
validation_data.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000
mean,28.048398,215.062427,428.592238,173.233107,295.895955,304.898293
std,14.624430,601.914503,1510.155974,554.991776,1104.872018,1235.991811
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,37.000000,48.000000,66.000000,27.000000,43.000000,31.000000
max,79.000000,11567.000000,25273.000000,8292.000000,19844.000000,22272.000000


In [26]:
validation_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [27]:
train_data.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

### Create deck, num, side columns

In [28]:
train_data[['Deck', 'Num', 'Side']] = train_data['Cabin'].str.split('/', expand=True)
validation_data[['Deck', 'Num', 'Side']] = validation_data['Cabin'].str.split('/', expand=True)

### Create passenger group column

In [29]:
train_data[['Group', 'Group_num']] = train_data['PassengerId'].str.split('_', expand=True)
validation_data[['Group', 'Group_num']] = validation_data['PassengerId'].str.split('_', expand=True)

### Create First and Second name calumns

In [30]:
train_data[['FirstName', 'SecondName']] = train_data['Name'].str.split(' ', expand=True)
validation_data[['FirstName', 'SecondName']] = validation_data['Name'].str.split(' ', expand=True)

### Convert the categorical columns

In [31]:
#train_data['HomePlanet'] = train_data['HomePlanet'].astype('category').cat.codes
#train_data['Destination'] = train_data['Destination'].astype('category').cat.codes
train_data['Deck'] = train_data['Deck'].astype('category').cat.codes
train_data['SecondName'] = train_data['SecondName'].astype('category').cat.codes


#validation_data['HomePlanet'] = validation_data['HomePlanet'].astype('category').cat.codes
#validation_data['Destination'] = validation_data['Destination'].astype('category').cat.codes
validation_data['Deck'] = validation_data['Deck'].astype('category').cat.codes
validation_data['SecondName'] = validation_data['SecondName'].astype('category').cat.codes

### Convert the integer columns

In [32]:
train_data['CryoSleep'] = train_data['CryoSleep'].astype('int')
train_data['Num'] = train_data['Num'].astype('int')
train_data['Group'] = train_data['Group'].astype('int')
train_data['Group_num'] = train_data['Group_num'].astype('int')
train_data['VIP'] = train_data['VIP'].astype('int')
train_data['Transported'] = train_data['Transported'].astype('int')


validation_data['CryoSleep'] = validation_data['CryoSleep'].astype('int')
validation_data['Num'] = validation_data['Num'].astype('int')
validation_data['Group'] = validation_data['Group'].astype('int')
validation_data['Group_num'] = validation_data['Group_num'].astype('int')
validation_data['VIP'] = validation_data['VIP'].astype('int')

### Apply one-hot encoding

In [33]:
train_data = pd.get_dummies(train_data, columns=['HomePlanet'])
train_data = pd.get_dummies(train_data, columns=['Destination'])
train_data = pd.get_dummies(train_data, columns=['Side'])

validation_data = pd.get_dummies(validation_data, columns=['HomePlanet'])
validation_data = pd.get_dummies(validation_data, columns=['Destination'])
validation_data = pd.get_dummies(validation_data, columns=['Side'])

### Create Underage column

In [34]:
train_data['Underage'] = train_data['Age'] < 18
validation_data['Underage'] = validation_data['Age'] < 18

### Create TotalSpent Column

In [35]:
train_data['TotalSpent'] = train_data['RoomService'] \
                            + train_data['FoodCourt'] \
                            + train_data['ShoppingMall'] + train_data['Spa'] \
                            + train_data['VRDeck']

validation_data['TotalSpent'] = validation_data['RoomService'] \
                            + validation_data['FoodCourt'] \
                            + validation_data['ShoppingMall'] + validation_data['Spa'] \
                            + validation_data['VRDeck']

In [223]:
train_data.head()

,PassengerId,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Side_P,Side_S,Underage,TotalSpent
0,0001_01,0,B/0/P,39.0,0,0.0,0.0,0.0,0.0,0.0,...,False,True,False,False,False,True,True,False,False,0.0
1,0002_01,0,F/0/S,24.0,0,109.0,9.0,25.0,549.0,44.0,...,True,False,False,False,False,True,False,True,False,736.0
2,0003_01,0,A/0/S,58.0,1,43.0,3576.0,0.0,6715.0,49.0,...,False,True,False,False,False,True,False,True,False,10383.0
3,0003_02,0,A/0/S,33.0,0,0.0,1283.0,371.0,3329.0,193.0,...,False,True,False,False,False,True,False,True,False,5176.0
4,0004_01,0,F/1/S,16.0,0,303.0,70.0,151.0,565.0,2.0,...,True,False,False,False,False,True,False,True,True,1091.0


In [37]:
train_data.columns

Index(['PassengerId', 'CryoSleep', 'Cabin', 'Age', 'VIP', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported',
       'Deck', 'Num', 'Group', 'Group_num', 'FirstName', 'SecondName',
       'HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars',
       'Destination_55 Cancri e', 'Destination_PSO J318.5-22',
       'Destination_TRAPPIST-1e', 'Side_P', 'Side_S', 'Underage',
       'TotalSpent'],
      dtype='object')

### Train model

In [213]:
from sklearn.model_selection import train_test_split

features = [
            #'PassengerId', 
            'CryoSleep', 
            #'Cabin', 
            'Age', 
            'VIP', 
            'RoomService',
            'FoodCourt', 
            'ShoppingMall', 
            'Spa', 
            'VRDeck', 
            #'Name', 
            #'Transported',
            'Deck', 
            #'Num', 
            #'Group', 
            #'Group_num', 
            #'FirstName', 
            #'SecondName',
            'HomePlanet_Earth', 
            'HomePlanet_Europa', 
            'HomePlanet_Mars',
            'Destination_55 Cancri e', 
            'Destination_PSO J318.5-22',
            'Destination_TRAPPIST-1e', 
            'Side_P', 
            'Side_S', 
            #'Underage',
            #'TotalSpent'
            ]

X = train_data[features]
Y = train_data.Transported

X_train, X_test, Y_train, Y_test = train_test_split(X,Y)


# Preparing val_data
X_val = validation_data[features]

In [214]:
print(len(X_train))
print(len(Y_train))
print(len(X_test))
print(len(Y_test))
print(len(X_val))

4954
4954
1652
1652
4277


# Visualizing Data

In [215]:
numeric_cols = train_data.select_dtypes(include=[np.number])
corr_matrix = numeric_cols.corr()

In [216]:
corr_matrix.style.background_gradient(cmap='coolwarm')

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Group,Group_num,SecondName,TotalSpent
CryoSleep,1.000000,-0.085701,-0.082303,-0.255398,-0.210782,-0.228612,-0.202216,-0.199095,0.462803,0.026156,-0.034108,-0.003173,0.066939,0.011613,-0.386646
Age,-0.085701,1.000000,0.098138,0.074783,0.135844,0.042314,0.123820,0.105031,-0.082553,-0.258787,-0.136822,-0.006416,-0.129544,0.015481,0.196001
VIP,-0.082303,0.098138,1.000000,0.029009,0.135386,0.013660,0.064792,0.130061,-0.042260,-0.196104,-0.107583,0.014938,-0.005312,-0.004108,0.165895
RoomService,-0.255398,0.074783,0.029009,1.000000,-0.013614,0.060478,0.012472,-0.026002,-0.247291,-0.025967,-0.010955,-0.003498,-0.028590,-0.002374,0.224410
FoodCourt,-0.210782,0.135844,0.135386,-0.013614,1.000000,-0.012320,0.215995,0.216997,0.055025,-0.330981,-0.186160,-0.006318,0.026838,0.010941,0.753124
ShoppingMall,-0.228612,0.042314,0.013660,0.060478,-0.012320,1.000000,0.022168,0.000383,0.011602,-0.035299,0.003917,0.020017,-0.039807,0.002776,0.216893
Spa,-0.202216,0.123820,0.064792,0.012472,0.215995,0.022168,1.000000,0.149447,-0.219854,-0.233928,-0.135590,-0.010564,0.020549,0.001146,0.592827
VRDeck,-0.199095,0.105031,0.130061,-0.026002,0.216997,0.000383,0.149447,1.000000,-0.207950,-0.261052,-0.142014,0.006962,0.008798,0.015701,0.575334
Transported,0.462803,-0.082553,-0.042260,-0.247291,0.055025,0.011602,-0.219854,-0.207950,1.000000,-0.112588,-0.046766,0.023216,0.075980,0.007787,-0.190992
Deck,0.026156,-0.258787,-0.196104,-0.025967,-0.330981,-0.035299,-0.233928,-0.261052,-0.112588,1.000000,0.538777,0.002316,-0.077438,-0.026139,-0.403596


# MLP

## Creating model

In [217]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score, classification_report, confusion_matrix

In [218]:
t_model = MLPClassifier(hidden_layer_sizes=(10, 10))

In [219]:
t_model.fit(X_train, Y_train)

MLPClassifier(hidden_layer_sizes=(10, 10))

In [220]:
predictions = t_model.predict(X_test)
predictions

array([1, 0, 0, ..., 1, 0, 1])

## Evaluating model

In [221]:
t_mae = mean_absolute_error(Y_test, predictions)
t_acc = accuracy_score(Y_test, predictions)
t_rep = classification_report(Y_test, predictions)
t_conf = confusion_matrix(Y_test, predictions)

In [222]:
#print(t_mae)
print(t_acc)
#print(t_rep)
#print(t_conf)

0.7966101694915254


In [224]:
predictions = t_model.predict(X_val)
predictions = predictions.astype(bool)

In [225]:
output = pd.DataFrame({'PassengerId': validation_data.PassengerId,
                       'Transported': predictions})
output.to_csv('submission.csv', index=False)